In [1]:
import random
import os
import glob
import shutil

In [2]:
random_country_dir = random.choice(os.listdir("../country_data"))
random_country_dir

'COM_Comoros'

In [3]:
figures = glob.glob(f"../country_data/{random_country_dir}/figures/*.png", recursive = True)
figures

['../country_data/COM_Comoros/figures/COM_CDIAC_1.png',
 '../country_data/COM_Comoros/figures/COM_BP_1.png',
 '../country_data/COM_Comoros/figures/COM_CO2_totals.png',
 '../country_data/COM_Comoros/figures/COM_EIA_1.png',
 '../country_data/COM_Comoros/figures/COM_GCP_1.png',
 '../country_data/COM_Comoros/figures/COM_Minx_top20_subsectors.png',
 '../country_data/COM_Comoros/figures/COM_CAIT_gases_1.png',
 '../country_data/COM_Comoros/figures/COM_UNFCCC_NAI_1.png']

In [4]:
random_fig = random.choice(figures)
random_fig

'../country_data/COM_Comoros/figures/COM_CO2_totals.png'

In [5]:
shutil.copyfile(random_fig, "../supplemental/random_daily_fig.png",)

'../supplemental/random_daily_fig.png'

In [6]:
print("random_fig path imported")

random_fig path imported
